<a href="https://colab.research.google.com/github/jhyoo78/jhyoo78/blob/main/next_word_prediction_Bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 인용 --> https://www.kaggle.com/code/ysthehurricane/next-word-prediction-bi-lstm-tutorial-easy-way

import os
import numpy as np

# 하나의 문장 내에서 다음 단어들을 맞출 때 import할 것들
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# 단어가 아닌 sentence 단위로 전처리할 경우 추가할 것
import nltk
nltk.download('punkt')   # ‘punkt= 문장 tokenizer’를 다운로드
from nltk.tokenize import word_tokenize, sent_tokenize
import re                # 파이썬 정규표현을 위한 표준 라이브러리임

In [ ]:
from google.colab import files     # Colab에서 local PC내의 log file 읽어 들이는 라이브러리
uploaded = files.upload()          # 파일을 여러 개 선택 가능함, 실험을 위해 학습 용 1개와 test 용 1개 로그 파일을 선택

In [ ]:
# 하나의 문장 내에서 다음 단어를 맞출때 사용...
input_sequence = []

f = open('test1.txt', 'r')          # 1,100 줄의 log 파일
lines = f.readlines()

tokenizer = Tokenizer(oov_token='<oov>')     #  토큰화할 때 적용할 파라미터 --> 사전에 없는 단어는 '<oov>' 로 출력
tokenizer.fit_on_texts(lines)                #  전체 단어의 index와 단어 벡터를 생성(Keras), #  단어들을 dictionary 로 변환. 예 --> {'<oov>': 1, 'systemd': 2, 'starting': 3, 'for': 4,......}
total_words = len(tokenizer.word_index) + 1
print("total words=", total_words)
print("word_index=", tokenizer.word_index)

for line in lines:
    #print(line)                                    # log msg를 한 줄씩 출력
    token_list = tokenizer.texts_to_sequences([line])[0]  # 각 문장(line)의 단어에 index를 부여해줌.
    print("token_list=", token_list)                      #  token_list = [3, 486, 299, 183], [3, 50, 300, 301].... 한 line에 하나의 list가 출력됨

    for i in range(1, len(token_list)):             #  한 문장 내의 단어 수만큼 반복
        n_gram_sequence = token_list[:i+1]
        #print("n_gram_seq=", n_gram_sequence)
        input_sequence.append(n_gram_sequence)     # --> [[3, 486], [3, 486, 299], [3, 486, 299, 183], [3, 50], [3, 50, 300], [3, 50, 300, 301],....]

print("input_sequence=", input_sequence)
print("Total input sequences: ", len(input_sequence))
f.close()

In [ ]:
# pad sequences, 문장의 길이가 같아지도록 패딩 처리
max_sequence_len = max([len(x) for x in input_sequence])
input_sequence = np.array(pad_sequences(input_sequence, maxlen=max_sequence_len, padding='pre'))

print(input_sequence)
print(input_sequence[1])
print("max_log_seq_length=", max_sequence_len)

In [ ]:
# create features and label
xs, labels = input_sequence[:,:-1],input_sequence[:,-1]
# [:, :] 는 [all rows, all columns]에 해당하며,  [:, :-1]은 all rows 와 마지막 column을 제외한 all columns에 해당함.
# [:, -1] 은 last column의 모든 rows에 해당함.

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)
# labels 내의 숫자를 각 행에 One-hot encoding해 준다. 즉 labels는 1차원 정수열이어야 한다.

print("xs and shape=", xs.shape)
print("xs=", xs[:10])    # len(input_seq) x padding
print("labels=", labels)
print("ys=", ys)
print("labels_shape=",labels.shape)
print("ys_shape=", ys.shape)

In [ ]:
# Bi- LSTM Neural Network Model training

model = Sequential()
model.add(Embedding(total_words, 200, input_length=max_sequence_len-1))  # vector 100 --> 200으로 변경
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(learning_rate =0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=10, verbose=1)  # epoch=50,   xs와 ys에 대해 학습을 시작함.
#print model.summary()
print(model)

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

# Predicting next word  

In [ ]:
seed_text = "finished apply"
next_words = 7    # 예측할 다음 단어 수...

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]      # 시험용 seed_text를 token화
    print(token_list)

    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')  # padding : 예 --> [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 9]]
    print("token_list=", token_list)

    y_prob = model.predict(token_list, verbose=0)    # model.predicted_classes 는 deprecated 되었으니 이 코드를 사용해야 함.
    print(y_prob, len(y_prob[0]))                    # total_words(1575개) 각각에 대한 확률을 출력함.  예 --> [[1.3384e-06 1.2275e-06 5.1810e-04 ... 3.392e-05]]

    predicted = y_prob.argmax(axis=-1)               # 가장 큰 확률 값을 갖는 단어의 index를 선택
                                                     # axis가 0 이면 2차원 행렬에서 열(가장 높은 차원), axis=1이면 행(다음으로 높은 차원), axis=-1이면 마지막 차원(가장 낮은 차원, 2차원에서는 행)을 기준 축으로 함
                                                     # argmax()는 가장 큰 값이 아닌 그 값의 인덱스 값을 반환함.
    print("predicted=", predicted, max(y_prob[0]))   # 예측된 단어의 index와  확률을 출력

    output_word = ""
    for word, index in tokenizer.word_index.items():   # 실제 단어로 변환하여 출력에 append 함.
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

# 끝...